# Homework and bake-off: Relation extraction using distant supervision

In [ ]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [1]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [2]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [3]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [4]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [5]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [6]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [7]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [9]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [10]:
featurizers = [simple_bag_of_words_featurizer]

In [11]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear', max_iter=200)

In [48]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.850      0.382      0.683        340       5716
author                    0.824      0.542      0.746        509       5885
capital                   0.567      0.179      0.395         95       5471
contains                  0.788      0.598      0.741       3904       9280
film_performance          0.789      0.565      0.731        766       6142
founders                  0.801      0.392      0.663        380       5756
genre                     0.622      0.165      0.400        170       5546
has_sibling               0.823      0.242      0.557        499       5875
has_spouse                0.881      0.325      0.656        594       5970
is_a                      0.688      0.221      0.484        497       5873
nationality               0.658      0.166      0.413        301       5677
parents     

Studying model weights might yield insights:

In [49]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.545 Córdoba
     2.436 Taluks
     2.431 Valais
     ..... .....
    -1.182 who
    -1.254 country
    -1.379 Spain

Highest and lowest feature weights for relation author:

     3.061 author
     2.309 by
     2.219 musical
     ..... .....
    -2.177 developed
    -2.277 directed
    -2.302 1774

Highest and lowest feature weights for relation capital:

     3.279 capital
     1.610 posted
     1.567 Airport
     ..... .....
    -1.496 ’
    -1.636 includes
    -1.685 Westminster

Highest and lowest feature weights for relation contains:

     2.082 third-largest
     2.017 within
     1.869 suburb
     ..... .....
    -2.460 who
    -2.649 2002
    -2.679 band

Highest and lowest feature weights for relation film_performance:

     4.279 starring
     3.578 alongside
     3.450 co-starring
     ..... .....
    -1.882 commonly
    -1.903 Aur
    -1.998 Westminster

Highest and lowest feature weights for relation founder

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [50]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [51]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [52]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)                        
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [53]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.837      0.438      0.708        340       5716
author                    0.835      0.436      0.706        509       5885
capital                   0.541      0.211      0.412         95       5471
contains                  0.659      0.406      0.586       3904       9280
film_performance          0.854      0.337      0.653        766       6142
founders                  0.783      0.237      0.536        380       5756
genre                     0.538      0.082      0.255        170       5546
has_sibling               0.787      0.253      0.553        499       5875
has_spouse                0.846      0.360      0.666        594       5970
is_a                      0.723      0.163      0.429        497       5873
nationality               0.691      0.216      0.480        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [54]:
def run_svm_model_factory():

    ##### YOUR CODE HERE
    from sklearn.svm import SVC
    
    svc_model_factory = lambda: SVC(kernel='linear')
    
    return rel_ext.experiment(splits, train_split='train', test_split='dev',
                             featurizers=featurizers, model_factory=svc_model_factory,
                             verbose=True)

In [55]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."

In [56]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.771      0.356      0.625        340       5716
author                    0.756      0.607      0.720        509       5885
capital                   0.632      0.253      0.486         95       5471
contains                  0.783      0.605      0.740       3904       9280
film_performance          0.739      0.621      0.712        766       6142
founders                  0.728      0.416      0.633        380       5756
genre                     0.544      0.253      0.442        170       5546
has_sibling               0.747      0.236      0.522        499       5875
has_spouse                0.827      0.347      0.648        594       5970
is_a                      0.633      0.264      0.494        497       5873
nationality               0.524      0.183      0.381        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [12]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word + subject_object_suffix] += 1 
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                             featurizers=[directional_bag_of_words_featurizer],
                             model_factory=model_factory, verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.877      0.421      0.721        340       5716
author                    0.831      0.578      0.764        509       5885
capital                   0.710      0.232      0.502         95       5471
contains                  0.806      0.670      0.774       3904       9280
film_performance          0.832      0.646      0.787        766       6142
founders                  0.815      0.405      0.678        380       5756
genre                     0.750      0.265      0.549        170       5546
has_sibling               0.874      0.251      0.584        499       5875
has_spouse                0.890      0.354      0.683        594       5970
is_a                      0.726      0.245      0.522        497       5873
nationality               0.691      0.223      0.486        301       5677
parents     

In [13]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [14]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [15]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        tags = get_tags(ex.middle_POS)
        for tag in get_tag_bigrams(tags):
            feature_counter[tag] += 1
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        tags = get_tags(ex.middle_POS)
        for tag in get_tag_bigrams(tags):
            feature_counter[tag] += 1

    return feature_counter


def get_tag_bigrams(tags):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"

    ##### YOUR CODE HERE
    result = []
    if len(tags) > 0:
        result = [start_symbol + ' ' + tags[0]]
        for index in range(len(tags) - 1):
            result.append(tags[index] + ' ' + tags[index + 1])
 
        result.append(tags[len(tags) - 1] + ' ' + end_symbol)
    
    return result 


def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                             featurizers=[middle_bigram_pos_tag_featurizer],
                             model_factory=model_factory, verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.847      0.374      0.676        340       5716
author                    0.735      0.316      0.581        509       5885
capital                   0.500      0.137      0.327         95       5471
contains                  0.757      0.594      0.718       3904       9280
film_performance          0.697      0.433      0.622        766       6142
founders                  0.533      0.168      0.372        380       5756
genre                     0.532      0.147      0.349        170       5546
has_sibling               0.669      0.170      0.422        499       5875
has_spouse                0.785      0.290      0.585        594       5970
is_a                      0.609      0.141      0.366        497       5873
nationality               0.396      0.063      0.193        301       5677
parents     

In [16]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [17]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [24]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        synsets = get_synsets(ex.middle_POS)
        for s in synsets:
            feature_counter[s] += 1
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        synsets = get_synsets(ex.middle_POS)
        for s in synsets:
            feature_counter[s] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]

    synsets = []
    ##### YOUR CODE HERE
    for w, t in wt:
        wnt = convert_tag(t)
        wns = wn.synsets(w, wnt)
        for s in wns:
            synsets.append(str(s))

    return synsets


def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                             featurizers=[synset_featurizer],
                             model_factory=model_factory, verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.810      0.326      0.625        340       5716
author                    0.761      0.432      0.661        509       5885
capital                   0.611      0.232      0.460         95       5471
contains                  0.778      0.585      0.730       3904       9280
film_performance          0.795      0.548      0.730        766       6142
founders                  0.733      0.389      0.623        380       5756
genre                     0.493      0.218      0.394        170       5546
has_sibling               0.771      0.216      0.510        499       5875
has_spouse                0.853      0.311      0.633        594       5970
is_a                      0.601      0.233      0.457        497       5873
nationality               0.470      0.130      0.308        301       5677
parents     

{'featurizers': [<function __main__.synset_featurizer(kbt, corpus, feature_counter)>],
 'vectorizer': DictVectorizer(),
 'models': {'adjoins': LogisticRegression(max_iter=200, solver='liblinear'),
  'author': LogisticRegression(max_iter=200, solver='liblinear'),
  'capital': LogisticRegression(max_iter=200, solver='liblinear'),
  'contains': LogisticRegression(max_iter=200, solver='liblinear'),
  'film_performance': LogisticRegression(max_iter=200, solver='liblinear'),
  'founders': LogisticRegression(max_iter=200, solver='liblinear'),
  'genre': LogisticRegression(max_iter=200, solver='liblinear'),
  'has_sibling': LogisticRegression(max_iter=200, solver='liblinear'),
  'has_spouse': LogisticRegression(max_iter=200, solver='liblinear'),
  'is_a': LogisticRegression(max_iter=200, solver='liblinear'),
  'nationality': LogisticRegression(max_iter=200, solver='liblinear'),
  'parents': LogisticRegression(max_iter=200, solver='liblinear'),
  'place_of_birth': LogisticRegression(max_iter=20

In [25]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [26]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [18]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM 
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: 0.728
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

#   1) Textual description of your system.
# I went experimenting with a couple of classifiers from sklearn and different features 
# as well as combination of those features.
#
#   Classifiers
# SVC # I tried using SVC with the recommented kernel='linear' and with the defailt RBF.
# SVC with RBF performed slightly better (0.03-0.05) than with linear kernel. This was 
# the case across all combinations of features that were used in the experiment. 
# This classifier with RBF performed the best with [directional_bag_of_words_featurizer, 
# middle_length_featurizer, middle_bigram_pos_tag_featurizer and synset_featurizer] 
# features delivering:
# macro-average             0.887 (precision)      0.268 (recall)      0.594 (F-Score)
#
# Nearest Neigbhors # As SVC with RBF performed better I thought to do an experiment with 
# Nearest Neigbhors. It performed much better than SVC and Logistic Regression for the recall
# but not very well for the precision. As we precision had a double weight this classifier 
# performed as expected the worst. Its performance only slightly improved with different
# features combinations. Its best performance was:
# macro-average             0.441 (precision)      0.554 (recall)      0.459 (F-Score)
#
# Logistic Regression # As in the homework exercises this classifier performed really well with
# different features. Its performance (F-Score) continued to increase with adding more features.
# This classifier performed the best with [directional_bag_of_words_featurizer, middle_length_featurizer,
# middle_bigram_pos_tag_featurizer and synset_featurizer] features delivering:
# macro-average             0.726 (precision)     0.414 (recall)      0.617 (F-Score) 
#
# MLP # The last classifier used in the experiments was Multi-layer perception classifier. 
# It required quite some fine-tuning of the L2 penalty value (regularisation term) and the
# learning rate. Eventually, good performance (F-Score) with acceptable (hours) exution speed 
# was achieved with L2 penalty = 0.05/0.1 and learning rate = 0.01. This classifier performed the best with 
# [directional_bag_of_words_featurizer, middle_length_featurizer, middle_bigram_pos_tag_featurizer]
# features. Surprisingly for me (I am not sure why) adding synset_featurizer feature reduced 
# slightly its performance. This model delivered:
# macro-average             0.767 (precision)      0.614 (recall)      0.728 (F-Score)
# macro-average             0.777 (precision)      0.579 (recall)      0.717 (F-Score)
#
#   Features
# I experimented with various features:
# - removing stop words from features using left, right and middle (reduced performance)
# - features with words from the outside context (reduced performance)
# - same as the previous one but directional (reduced performance)
# - tried combitaion of the previous point with POS fearures (reduced performance)
# - bigrams and trigrams of context words (reduced performance)
# - mentions of the entities (no effect on performance due to very small presense)
# - glove middle (right and/or left) (SO and/or OS) words featuriser (didn't deliver better 
# performance with any of the models)
# + lenght of the middle (increased performance)
# + directional bag of middle words (increased performance)
# + bigram pos tags from the middle text (increased performance)
# +/- synsets (increased performance except for MLP). This featuriser also significantly 
# increased the execution time so if no significant performance improvement was observed
# it wasn't practical to use it
#
#
#   Original System
# It is a very simple system. The solution uses MLP (L2 penalty value = 0.05 and 
# leanring rate = 0.01) with three features:
# - directional bag of words from the middle (context between the entities)
# - bigram POS tags from the middle 
# - length of the middle
# The system reuses the code from the homework of the directional_bag_of_words_featurizer
# and middle_bigram_pos_tag_featurizer functions, and adds a new middle_length_featurizer one.
#
#   2) The code for your original system.

from sklearn.neural_network import MLPClassifier

def middle_length_featurizer(kbt, corpus, feature_counter):
    # Featurises the length of the context between the two entities
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[str(len(ex.middle))] += 1 
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[str(len(ex.middle))] += 1

    return feature_counter

def run_model(splits, train_split='tiny', test_split='dev', verbose=False):
    mlpc_model_factory = lambda: MLPClassifier(
        alpha=0.05, learning_rate_init=0.01, max_iter=500)
    
    featurisers = [directional_bag_of_words_featurizer,
                   middle_length_featurizer,
                   middle_bigram_pos_tag_featurizer]
    
    return rel_ext.experiment(splits, train_split=train_split, test_split=test_split, 
                              featurizers=featurisers,
                              model_factory=mlpc_model_factory,
                              verbose=verbose)

# STOP COMMENT: Please do not remove this comment.

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [21]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    rel_ext_data_home_test = os.path.join(rel_ext_data_home, 'bakeoff-rel_ext-test-data')
       
    bakeoff_results = run_model(splits, 
                                train_split='train', 
                                test_split='dev')
    
    rel_ext.bake_off_experiment(bakeoff_results, rel_ext_data_home_test)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.827      0.664      0.788        438       7122
author                    0.866      0.800      0.852        645       7329
capital                   0.689      0.539      0.653        115       6799
contains                  0.799      0.761      0.791       3808      10492
film_performance          0.823      0.761      0.810       1011       7695
founders                  0.781      0.651      0.751        444       7128
genre                     0.614      0.431      0.566        188       6872
has_sibling               0.825      0.584      0.762        717       7401
has_spouse                0.875      0.671      0.824        780       7464
is_a                      0.756      0.548      0.703        611       7295
nationality               0.660      0.648      0.657        383       7067
parents     

In [22]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported
# by the code above. Please enter only a number between
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
    0.726
